In [2]:
import os
from glob import glob
from PIL import Image

import pandas as pd
import numpy as np
import tensorflow as tf
from sklearn.model_selection import train_test_split
from tensorflow.keras.utils import to_categorical # convert to one-hot-encoding

from tensorflow.keras.preprocessing.image import ImageDataGenerator
from keras.models import Sequential
from keras.layers import Dense, Dropout, Flatten, Conv2D, MaxPool2D
from tensorflow.keras.optimizers import SGD
from tensorflow.keras.callbacks import ReduceLROnPlateau, EarlyStopping
from tensorflow.keras import backend as K

%matplotlib inline
import matplotlib.pyplot as plt

In [3]:
X_train = np.load("../input/256-192-dataset/256_192_train.npy")
y_train = np.load("../input/256-192-dataset/train_labels.npy")
X_val = np.load("../input/256-192-dataset/256_192_val.npy")
y_val = np.load("../input/256-192-dataset/val_labels.npy")

In [4]:
train_datagen = ImageDataGenerator(
        featurewise_center=False,  # set input mean to 0 over the dataset
        samplewise_center=False,  # set each sample mean to 0
        featurewise_std_normalization=False,  # divide inputs by std of the dataset
        samplewise_std_normalization=False,  # divide each input by its std
        zca_whitening=False,  # apply ZCA whitening
        rotation_range=90,  # randomly rotate images in the range (degrees, 0 to 180)
        zoom_range = 0.1, # Randomly zoom image 
        width_shift_range=0.1,  # randomly shift images horizontally (fraction of total width)
        height_shift_range=0.1,  # randomly shift images vertically (fraction of total height)
        horizontal_flip=True,  # randomly flip images
        vertical_flip=True,  # randomly flip images
        shear_range = 10)

train_datagen.fit(X_train)

val_datagen = ImageDataGenerator()
val_datagen.fit(X_val)

In [5]:
y_train = to_categorical(y_train)
y_val = to_categorical(y_val)

In [7]:
input_shape = (192, 256, 3)
num_classes = 7

#Keras Sequential API
#CNN Model: Conv x2 -> MaxPool -> Dropout -> Conv x2 -> MaxPool -> Dropout -> Flatten -> Dense -> Dropout -> Dense 
model = Sequential()
model.add(Conv2D(32, kernel_size=(3, 3), activation='relu', padding = 'Same', input_shape=input_shape))
model.add(Conv2D(32, kernel_size=(3, 3), activation='relu', padding = 'Same'))
model.add(MaxPool2D(pool_size = (2, 2)))
model.add(Dropout(0.25))

model.add(Conv2D(64, (3, 3), activation='relu', padding = 'Same'))
model.add(Conv2D(64, (3, 3), activation='relu', padding = 'Same'))
model.add(MaxPool2D(pool_size=(2, 2)))
model.add(Dropout(0.40))

model.add(Flatten())
model.add(Dense(128, activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(num_classes, activation='softmax'))

model.summary()
# Configure and compile the model

optimizer=tf.keras.optimizers.SGD(
    learning_rate=0.01, momentum=0.9, nesterov=False, name="SGD")
model.compile(loss='CategoricalCrossentropy',
              optimizer=optimizer,
            metrics=['accuracy'])

Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_4 (Conv2D)            (None, 192, 256, 32)      896       
_________________________________________________________________
conv2d_5 (Conv2D)            (None, 192, 256, 32)      9248      
_________________________________________________________________
max_pooling2d_2 (MaxPooling2 (None, 96, 128, 32)       0         
_________________________________________________________________
dropout_3 (Dropout)          (None, 96, 128, 32)       0         
_________________________________________________________________
conv2d_6 (Conv2D)            (None, 96, 128, 64)       18496     
_________________________________________________________________
conv2d_7 (Conv2D)            (None, 96, 128, 64)       36928     
_________________________________________________________________
max_pooling2d_3 (MaxPooling2 (None, 48, 64, 64)       

In [8]:
learning_rate_reduction = ReduceLROnPlateau(monitor='val_acc', patience=3, verbose=1, factor=0.5, 
                                            min_lr=0.000001, cooldown=2)

In [ ]:
batch_size = 64
epochs = 50
history = model.fit_generator(train_datagen.flow(X_train,y_train, batch_size=batch_size),
                              epochs = epochs, validation_data = val_datagen.flow(X_val, y_val),
                              verbose = 1, steps_per_epoch=(X_train.shape[0] // batch_size),
                              validation_steps=(X_val.shape[0] // batch_size),
                              callbacks=[learning_rate_reduction])

/opt/conda/lib/python3.7/site-packages/keras/engine/training.py:1972: UserWarning: `Model.fit_generator` is deprecated and will be removed in a future version. Please use `Model.fit`, which supports generators.
  warnings.warn('`Model.fit_generator` is deprecated and '
2022-04-04 01:17:30.249862: I tensorflow/compiler/mlir/mlir_graph_optimization_pass.cc:185] None of the MLIR Optimization Passes are enabled (registered 2)


Epoch 1/50


2022-04-04 01:17:32.222604: I tensorflow/stream_executor/cuda/cuda_dnn.cc:369] Loaded cuDNN version 8005


126/126 [==============================] - 92s 669ms/step - loss: 1.1333 - accuracy: 0.6622 - val_loss: 1.0204 - val_accuracy: 0.6585
Epoch 2/50
126/126 [==============================] - 83s 662ms/step - loss: 1.0197 - accuracy: 0.6680 - val_loss: 0.9939 - val_accuracy: 0.6652
Epoch 3/50
126/126 [==============================] - 83s 658ms/step - loss: 0.9887 - accuracy: 0.6687 - val_loss: 0.9827 - val_accuracy: 0.6607
Epoch 4/50
126/126 [==============================] - 84s 666ms/step - loss: 0.9736 - accuracy: 0.6699 - val_loss: 0.9617 - val_accuracy: 0.6585
Epoch 5/50
126/126 [==============================] - 84s 667ms/step - loss: 0.9797 - accuracy: 0.6681 - val_loss: 0.9524 - val_accuracy: 0.6629
Epoch 6/50
126/126 [==============================] - 83s 655ms/step - loss: 0.9666 - accuracy: 0.6699 - val_loss: 0.9314 - val_accuracy: 0.6696
Epoch 7/50
126/126 [==============================] - 83s 660ms/step - loss: 0.9629 - accuracy: 0.6689 - val_loss: 0.9417 - val_accuracy: 0.6

In [ ]:
loss_val, acc_val = model.evaluate(X_val, y_val, verbose=1)
print("Validation: accuracy = %f  ;  loss_v = %f" % (acc_val, loss_val))

In [ ]:
X_test = np.load("../input/256-192-dataset/256_192_test.npy")
y_test = np.load("../input/256-192-dataset/test_labels.npy")

y_test = to_categorical(y_test)


test_generator = test_datagen.flow(X_test, y_test)

In [ ]:
test_loss, test_accuracy= model.evaluate_generator(test_generator, verbose=1)
print("Test: accuracy = %f  ;  loss = %f " % (test_accuracy, test_loss))

In [ ]:
from tensorflow.keras.models import save_model
filepath = './own_model.h5'
tf.keras.models.save_model(
    model, filepath, overwrite=True, include_optimizer=True, save_format='h5',
    signatures=None, options=None, save_traces=True
)

In [ ]:
y_pred=model.predict_generator(X_test)

In [ ]:
from sklearn.metrics import confusion_matrix
from sklearn.metrics import precision_score, recall_score, f1_score, accuracy_score
import matplotlib.pyplot as plt
K.int_shape(y_pred)

In [ ]:
y_pred=np.argmax(y_pred, axis=1)
y_test=np.argmax(y_test, axis=1)
cm = confusion_matrix(y_test, y_pred)
print(cm)

In [ ]:
target_names = ['class 0', 'class 1', 'class 2', 'class 3', 'class 4', 'class 5', 'class 6']

In [ ]:
from sklearn.metrics import classification_report
print(classification_report(y_test, y_pred, target_names=target_names))

In [ ]:
# summarize history for accuracy
print(history.history.keys())
plt.plot(history.history['acc'])
plt.plot(history.history['val_acc'])
#plt.plot(history.history['test_accuracy'])
plt.title('model accuracy')
plt.ylabel('accuracy')
plt.xlabel('epoch')
plt.legend(['train', 'validation','test'], loc='upper left')
plt.show()
# summarize history for loss
plt.plot(history.history['loss'])
plt.plot(history.history['val_loss'])
#plt.plot(history.history['test_loss'])
plt.title('model loss')
plt.ylabel('loss')
plt.xlabel('epoch')
plt.legend(['train', 'validation','test'], loc='upper left')
plt.show()


In [ ]:
from imblearn.metrics import sensitivity_specificity_support
from sklearn.metrics import roc_auc_score
from sklearn.metrics import roc_curve
result = sensitivity_specificity_support(y_test,y_pred,average='macro' )

print("Sensitivity: {:5.2f}%".format(100*result[0]), "specificity {:5.2f}%".format(100*result[1]))

In [ ]:
# roc curve for classes
fpr = {}
tpr = {}
thresh ={}

n_class = 7

for i in range(n_class):    
    fpr[i], tpr[i], thresh[i] = roc_curve(y_test, y_pred, pos_label=i)
    
# plotting

colors=['blue','red','green','yellow','orange','pink','black']

for i in range(0,7):
    plt.plot(fpr[i], tpr[i], linestyle='--',color=colors[i], label='Class'+ str(i) +'vs Rest')


plt.title('Multiclass ROC curve')
plt.xlabel('False Positive Rate')
plt.ylabel('True Positive rate')
plt.legend(loc='best')
plt.savefig('Multiclass ROC',dpi=300);